# Advanced Human Behavior Simulation Framework

<div class="admonition note">
    <p class="admonition-title">In progress</p>
    <p>This document is a work in progress if you see any errors, or exclusions or have any problems, please <a href="https://github.com/absespy/ABSESpy/issues">get in touch with us</a>.</p>
</div>

In the dynamic realm of Socio-Ecological Systems (SES) modeling, simulating human behavior is paramount. Humans, as key actors, play a pivotal role in shaping the interactions and outcomes within SES. Recognizing the complexity and nuances of human actions, ABSESpy introduces an advanced framework to model intricate human behaviors effectively.

<div align="center">
	<img src="https://songshgeo-picgo-1302043007.cos.ap-beijing.myqcloud.com/uPic/MoHuB.png" alt="Drawing" style="width: 400px;"/>
</div>

The beauty of ABSESpy lies in its framework that seamlessly integrates these user-defined functions, creating a smooth workflow that mirrors the intricacies of human behavior in the real world. When implementing this human behavior framework with ABSESpy, users are allowed to define key elements of decision-making step by step with the following concepts:

- **Decision**: First, define the types of decisions that an actor can make. A particular type of decision can be made by various types of actors. However, pre-defining what the decision types are helps users to better understand and organize their models.
- **Perception**: Actors make decisions based on their perception of the surrounding world, including both the natural and social environments.
- **Evaluate**: As the process of evaluating perceptions can vary, different actors may have different methods of evaluating decisions. Users can choose different evaluation methods for the same decision across various actors.
- **Response**: Finally, based on the decision-making process, actors respond and their actions impact the surrounding environment.

Let's dive into them.

## Decisions

In [1]:
from abses import Actor
from abses.actor import perception
from abses.decision import Decision
from abses.main import MainModel
import numpy as np


class OverWorking(Decision):
    """内卷策略"""

    __strategies__ = {
        True: "Decide to work longer.",
        False: "No longer work more...",
    }

    def setup(self):
        """Initial strategy of a new actor."""
        return np.random.choice([True, False], p=[0.1, 0.9])

A custom decision needs to inherit the `Decision` class, and users can customize the class attribute  `__strategies__` to set the choice of strategies and explanations for each strategy. The strategy does not need to be actively instantiated because a decision always requires a decision-making subject, which we will define next:

In [2]:
class InvolutingActor(Actor):
    """A poor guy who has to work harder..."""

    __decisions__ = OverWorking


model = MainModel()
actor = InvolutingActor(model=model)

As you can see, it's very simple. When customizing the main class, all you need to do is set `__decisions__` in the class attributes. This type of main actors will automatically have this decision when initialized. You can access it using `decisions` or a shortcut `d`:

In [3]:
actor.decisions.over_working

<over_working: False>

In [4]:
actor.d.over_working

<over_working: False>

## Preception

<!-- 通常来说，社会-生态系统中的行动者（即主体）在做出决策时，需要基于其对世界的感知 -->

Generally speaking, actors (i.e., agents) in socio-ecological systems (SES) need to base their decisions on their perception of the world.

<!-- 让我们来创建一个疲于工作的主体，它最初只会每天工作7小时。他会感知周围同事的工作强度。 -->
Let's create a agent who is tired from work, initially only working 7 hours a day. He will perceive the intensity of his colleagues' work around him.

In [5]:
class InvolutingActor(Actor):
    """A poor guy who has to work harder..."""

    __decisions__ = OverWorking

    def __init__(self, *args, working_hrs: float = 7.0, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.working_hrs = working_hrs
        self.overwork = False

    @perception
    def avg_working_hrs(self) -> float:
        """The average wealth of acquaintances."""
        colleagues = self.link.get("colleague")
        return colleagues.array("working_hrs").mean()

Now, let's create some agents and make them be colluague of our poor guy.

We use `link_to` method to do so. The tutorial of this method can be found [here](links.ipynb).

In [6]:
agents = model.agents.new(InvolutingActor, 3)
poor_guy, colleague_1, colleague_2 = agents

poor_guy.working_hrs = 7
colleague_1.working_hrs = 8
colleague_2.working_hrs = 8

# link_to() method can build links between agents and/or cells
poor_guy.link.to(colleague_1, "colleague")
poor_guy.link.to(colleague_2, "colleague")

poor_guy.link.get("colleague")

<ActorsList: (2)InvolutingActor>

In [7]:
# report the average working hours of its colleagues.
poor_guy.avg_working_hrs()

8.0

<!-- 这个主体感受到感知到同事们的平均工作时长之后，他就会进行评估。我们可以使用先前创建的决策 `OverWork` 中提供的装饰器类方法 `making` 来装饰评估函数。这样以来，这个函数的输出就会作为该决策的当前策略。 -->

After this entity perceives the average working hours of colleagues, it will conduct an evaluation. We can use the decorator class method `making` provided in the previously created decision `OverWork` to decorate the evaluation function. In this way, the output of this function will serve as the current strategy for that decision.

```python
class InvolutingActor(Actor):
    """A poor guy who has to work harder..."""

    # ... other methods ...

    @OverWorking.making
    def feel_peer_pressure(self):
        """Feel stressful when others work harder than self."""
        return self.working_hrs <= self.avg_working_hrs()
```

## Decision making (Responses)

<!-- 当主体产生了一个决策后，就会作出响应（即实际意义上的“行为”）。我们提供了另一个装饰器，用户使用该装饰器，可以更有逻辑地指定哪个决策在哪个策略下产生什么样的行为。-->

Once a decision is made by the subject, a response (i.e., "action" in practical terms) will be generated. We have provided another decorator `response`, and users can use this decorator to logically specify what kind of action is produced under which strategy for which decision. It determines how agents respond to their decisions. This can involve moving to a new location, altering their state, or interacting with other entities. Here, let our class add a simple response when it "feels peer pressure" -i.e., work one hour more...:

```python
class InvolutingActor(Actor):
    """A poor guy who has to work harder..."""

    # ... other methods ...

    @response(decision="over_working", strategy=True)
    def work_harder(self):
        """Work harder."""
        self.working_hrs += 1
```


<!-- 将上述主体的方法整合起来，我们会得到这样一个自定义的主体 `InvolutingActor` (用中文来说是"内卷的人"，即因为竞争压力不得不消耗自己精力，但没有实质进步的人。)：-->

Integrating the methods of the above-mentioned methods, we will get such a custom actor (agent) class `InvolutingActor` (in Chinese it is called "内卷的人", that is, a person who has to consume his own energy due to competitive pressure but does not make substantial progress.)

In [8]:
class InvolutingActor(Actor):
    """A poor guy who has to work harder..."""

    __decisions__ = OverWorking

    def __init__(self, *args, working_hrs: float = 7.0, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.working_hrs = working_hrs
        self.overwork = False

    @perception
    def avg_working_hrs(self) -> float:
        """The average wealth of acquaintances."""
        acquaintance = self.link.get("colleague")
        return acquaintance.array("working_hrs").mean()

    @OverWorking.response(strategy=True)
    def work_harder(self):
        """Work harder."""
        self.working_hrs += 1

    @OverWorking.making
    def feel_peer_pressure(self):
        """Feel stressful when others work harder than self."""
        return self.working_hrs <= self.avg_working_hrs()

<!-- 让我们看看现在发生了什么？我们可以使用 actor 的 `decision.making()` 方法，这样主体会自动完成「感知->评估->决策」的框架。即这个内卷的人自动增加了工作小时数。 -->
Let's see what's happening now? We can use the `decision.making()` method of the actor, so that the subject will automatically complete the "perception->evaluation->response" framework. That is, this person involved in intense competition has automatically increased his working hours (from 7 to 8).

In [9]:
agents = model.agents.new(InvolutingActor, 3)
poor_guy, colleague_1, colleague_2 = agents

poor_guy.working_hrs = 7
colleague_1.working_hrs = 8
colleague_2.working_hrs = 7

# link_to() method can build links between agents and/or cells
poor_guy.link.to(colleague_1, "colleague")
poor_guy.link.to(colleague_2, "colleague")

# decision making...
poor_guy.decisions.making()

In [10]:
poor_guy.working_hrs

8

<!-- 使用 `decisions.making` 会自动检查主体所有被设置的决策，并实践“感知->评估->响应” ("perception->evaluation->response") 框架。当这个决策结果不符合产生反应的条件时，则不会响应。看，当这个可怜的人工作时长不再比周围人的平均值低的时候，他不再主动增加自己的工作时长： -->

Using `decisions.making` will automatically check all the decisions set by the subject and implement the "perception->evaluation->response" framework. If the decision result does not meet the conditions for generating a response, it will not respond. Look, when this poor person's working hours are no longer lower than the average of those around him, he no longer actively increases his own working hours from 8 to 9:

In [11]:
poor_guy.decisions.making()
poor_guy.working_hrs

8